Download the Census Dissemination Block geometry and population and dwelling counts. 

In [1]:
import os
import requests

if not os.path.exists('./Data'):
    os.mkdir('./Data')
if not os.path.exists('./Data/DB_attr'):
    os.mkdir('./Data/DB_attr')
if not os.path.exists('./Data/DB_shape'):
    os.mkdir('./Data/DB_shape')
if not os.path.exists('./Data/StopLocations'):
    os.mkdir('./Data/StopLocations')
    
download = requests.get('http://www12.statcan.gc.ca/census-recensement/2016/geo/ref/gaf/files-fichiers/2016_92-151_XBB_csv.zip')
with open('./Data/DB_attr/DB_attr_2016.zip', 'wb') as f:
    f.write(download.content)
    
download = requests.get('http://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/files-fichiers/2016/ldb_000b16a_e.zip')
with open('./Data/DB_shape/DB_shape_2016.zip', 'wb') as f:
    f.write(download.content)
    
download = requests.get('https://bctransit.com/servlet/bctransit/data/GTFS%20-%20Victoria')
with open('./Data/StopLocations/StopLocations.zip', 'wb') as f:
    f.write(download.content)

Extract the zip files into their data folders.

In [2]:
from zipfile import ZipFile

with ZipFile('./Data/DB_attr/DB_attr_2016.zip', 'r') as zip_file:
    zip_file.extractall('./Data/DB_attr')
    
with ZipFile('./Data/DB_shape/DB_shape_2016.zip', 'r') as zip_file:
    zip_file.extractall('./Data/DB_shape')
    
with ZipFile('./Data/StopLocations/StopLocations.zip', 'r') as zip_file:
    zip_file.extractall('./Data/StopLocations')

Filter the DB Boundary shapefile to only include BC. Using the shapefile library allows us to lazy load the shapefile features making it far more memory efficient.

In [ ]:
import shapefile

reader = shapefile.Reader('./Data/DB_shape/ldb_000b16a_e.shp')

writer  = shapefile.Writer()
writer.fields = reader.fields[1:]

shape_records = reader.iterShapeRecords()

for shape_record in shape_records:
    if shape_record.record[3] == '59':
        writer.record(*shape_record.record)
        writer._shapes.append(shape_record.shape)
writer.save('./Data/DB_shape/BC_DB_shape_2016.shp')

Load the spatial data to a GeoDataFrame using GeoPandas

In [ ]:
import geopandas as gpd

db_shapes = gpd.read_file('./Data/DB_shape/BC_DB_shape_2016.shp')

The Stats Canada attribute data is quite large so we load it in chunks and also filter it to include only BC

In [ ]:
import pandas as pd
bc_db_data = pd.DataFrame()
canada_db_data_chunks = pd.read_csv("Data/DB_attr/2016_92-151_XBB.csv", 
                             chunksize=5000, encoding='latin1')

for chunk in canada_db_data_chunks:
    bc_records = chunk.loc[chunk[' PRuid/PRidu']==59]
    if not bc_db_data.empty and len(bc_records) > 0:
        bc_db_data = bc_db_data.append(bc_records)
    elif len(bc_records) > 0:
        bc_db_data = bc_records

bc_db_data.to_csv('./Data/DB_attr/BC_DB_attr.csv')